## Check your connection to Generative AI Hub


☝️ In the init_env.py file the values from `generative-ai-codejam/.aicore-config.json` are assigned to environmental variables. That way the Generative AI Hub [Python SDK](https://pypi.org/project/generative-ai-hub-sdk/) will connect to Generative AI Hub.

👉 For the Python SDK to know which resource group to use, you also need to set the `resource group` in the [`variables.py`](variables.py) file to your own `resource group` (e.g. **team-01**) that you created in the SAP AI Launchpad in exercise [00-connect-AICore-and-AILaunchpad](00-connect-AICore-and-AILaunchpad.md).

In [1]:
import init_env
import variables
import importlib
variables = importlib.reload(variables)

# TODO: You need to specify which model you want to use. In this case we are directing our prompt
# to the openAI API directly so you need to pick one of the GPT models. Make sure the model is actually deployed
# in genAI Hub. You might also want to chose a model that can also process images here already. 
# E.g. 'gpt-4.1-mini'
MODEL_NAME = 'gpt-4.1-mini'
# Do not modify the `assert` line below
assert MODEL_NAME!='', """You should change the variable `MODEL_NAME` with the name of your deployed model (like 'gpt-4o-mini') first!"""

init_env.set_environment_variables()
# Do not modify the `assert` line below 
assert variables.RESOURCE_GROUP!='', """You should change the value assigned to the `RESOURCE_GROUP` in the `variables.py` file to your own resource group first!"""
print(f"Resource group is set to: {variables.RESOURCE_GROUP}")

Resource group is set to: default


## Prompt an LLM in the Generative AI Hub...

...using OpenAI native client integration: https://help.sap.com/doc/generative-ai-hub-sdk/CLOUD/en-US/_reference/gen_ai_hub.html#openai

To understand the API and structures, check OpenAI documentation: https://platform.openai.com/docs/guides/text?api-mode=chat

In [2]:
from gen_ai_hub.proxy.native.openai import chat
from IPython.display import Markdown

messages = [
    {
        "role": "user", 
        "content": "What is the underlying model architecture of an LLM? Explain it as short as possible."
    }
]

kwargs = dict(model_name=MODEL_NAME, messages=messages)
response = chat.completions.create(**kwargs)
display(Markdown(response.choices[0].message.content))

An LLM (Large Language Model) is typically based on the Transformer architecture, which uses layers of self-attention and feed-forward neural networks to process and generate language by capturing contextual relationships in sequential data.

## Understanding roles
Most LLMs have the roles `system`, `assistant` (GPT) or `model` (Gemini) and `user` that can be used to steer the models response. In the previous step you only used the role `user` to ask your question. 

👉 Try out different `system` messages to change the response. You can also tell the model to not engage in smalltalk or only answer questions on a certain topic. Then try different user prompts as well!

Please note, that in OpenAI API with `o1` models and newer, `developer` messages replace the previous `system` messages.

In [3]:
messages = [
    {   "role": "system", 
        # TODO try changing the system prompt
        "content": "Speak like a Pirate."
    }, 
    {
        "role": "user", 
        "content": "What is the underlying model architecture of an LLM? Explain it as short as possible."
    }
]

kwargs = dict(model_name=MODEL_NAME, messages=messages)
response = chat.completions.create(**kwargs)
display(Markdown(response.choices[0].message.content))

Arrr! The secret sea chart of an LLM be the Transformer architecture — a mighty fleet o’ layers usin’ attention mechanisms to weigh every word in the treasure map (text) and steer the ship toward smart, crafty responses!

👉 Also try to have it speak like a pirate.

👉 Now let's be more serious! Tell it to behave like an SAP consultant talking to AI Developers.

👉 Ask it to behave like an SAP Consultant talking to ABAP Developers and to make ABAP comparisons.

## Hallucinations
👉 Run the following question.

In [4]:
messages = [
    {   "role": "system", 
        "content": "You are an SAP Consultant."
    }, 
    {
        "role": "user", 
        "content": "How does the data masking of the orchestration service work?"
    }
]

kwargs = dict(model_name=MODEL_NAME, messages=messages)
response = chat.completions.create(**kwargs)
display(Markdown(response.choices[0].message.content))

In SAP Integration Suite (orchestration service), data masking is used to protect sensitive information during message processing and monitoring. Here’s how data masking works in the orchestration service:

1. **Configuration of Data Masking Rules**:  
   You define data masking rules that specify which elements or fields in the message payload or headers should be masked. These rules are based on XPath expressions or JSONPath to precisely identify sensitive data locations within the message structure.

2. **Masking during Message Processing**:  
   When a message passes through the orchestration (integration flow), the service applies these masking rules to the message content, replacing sensitive data with masked values (e.g., asterisks or generic placeholder strings). This occurs before the message is stored in message monitoring or logs.

3. **Data Masking in Message Monitoring and Logs**:  
   The primary purpose of data masking is to avoid sensitive information exposure in message monitoring tools, logs, and tracing. The original data remains intact during runtime but is masked in the UI and logs where users can access the messages, ensuring compliance with data privacy and security standards.

4. **Scope of Masking**:  
   Masking can be applied to various data types, including XML, JSON, and plain text messages. This gives flexibility to secure data across different integration scenarios.

5. **No Impact on Message Processing**:  
   The masking is non-intrusive for the actual processing; it only affects how data is displayed in monitoring or logging interfaces but does not alter the message payload that is sent to the target system.

**Summary**: The orchestration service applies configured data masking rules to sensitive fields within messages during processing, masking them in logs and monitoring views without modifying the actual message content sent downstream. This ensures sensitive data protection and compliance with data privacy regulations.

If you want, I can provide details on how to configure these masking rules or where to manage them in the SAP Integration Suite.

☝️ Compare the response to [SAP Help - Generative AI Hub SDK](https://help.sap.com/doc/generative-ai-hub-sdk/CLOUD/en-US/_reference/orchestration-service.html). 

👉 What did the model respond? Was it the truth or a hallucination?

👉 Which questions work well, which questions still do not work so well?

# Use Multimodal Models

Multimodal models can use different inputs such as text, audio and [images](https://platform.openai.com/docs/guides/images-vision?api-mode=chat&format=base64-encoded#giving-a-model-images-as-input). In Generative AI Hub on SAP AI Core you can access multiple multimodal models (e.g. `gpt-4o-mini`).

👉 If you have not deployed one of the gpt-4o-mini models in previous exercises, then go back to the model library and deploy a model that can also process images.

👉 Now run the code snippet below to get a description for [the image of the AI Foundation Architecture](documents/ai-foundation-architecture.png). These descriptions can then for example be used as alternative text for screen readers or other assistive tech.

👉 You can upload your own image and play around with it!

In [5]:
import base64

# get the image from the documents folder
with open("documents/ai-foundation-architecture.png", "rb") as image_file:
    image_data = base64.b64encode(image_file.read()).decode('utf-8')

messages = [{
            "role": "user", 
             "content": [
                {"type": "text", 
                 "text": "Describe the images as an alternative text."},
                {"type": "image_url", 
                 "image_url": {
                    "url": f"data:image/png;base64,{image_data}"
                    }
                }
            ]
        }]

kwargs = dict(model_name=MODEL_NAME, messages=messages)
response = chat.completions.create(**kwargs)

display(Markdown(response.choices[0].message.content))

The image is a detailed architecture diagram titled "Single-tenant application (Retrieval Augmented Generation & Generative AI on SAP BTP)." It illustrates the components and interactions within SAP BTP (Business Technology Platform) for building an AI-enhanced single-tenant application.

- On the left, a User icon connects to "Application Clients" representing Mobile/Desktop devices.
- The application clients connect to the "HTML5 App Repository" and then to an "App Router" within a "Subaccount" labeled "Multi-Cloud."
- The Subaccount contains several components:
   - "User Interface" with SAPUI5 and UI5 Web Components listed.
   - "SAP Authorization and Trust Management service" managing trust relationships (represented with green arrows labeled "Trust").
   - "SAP Continuous Integration and Delivery" and "SAP Business Application Studio" icons below.
   - The "SAP Cloud Application Programming Model (CAP)" box in pink includes an "Application Service" with Use Case Logic, Data Management, LLM Plugins & SDKs, and SAP Cloud SDK for AI.
   - A connection heads to "SAP HANA Cloud" with "Vector Engine" highlighted.
   - The "Generative AI Hub" box in pink includes:
       - "SAP AI Launchpad"
       - "SAP AI Core" with Trust & Control and Prompt Registry
       - "Orchestration" with Grounding, Templating, Data Masking, and I/O Filtering
       - "Foundation Model Access" indicating "Partner built" (active) and "SAP built" (grayed out)
       - "Foundation Models" labeled SAP hosted.
- The Generative AI Hub connects via HTTPS to various external entities under "NETWORK":
   - "SAP On-Premise Solutions" using Cloud Connector.
   - "3rd Party Applications"
   - "SAP Cloud Solutions"
- The diagram includes numbered callouts (1, 2, 3) depicting flow between components:
   1. From Application Clients to CAP and then to Generative AI Hub (using Harmonized API).
   2. From CAP to SAP HANA Cloud.
   3. From Generative AI Hub back to CAP.
- A legend explains icons: Blue circles represent Access, Green arrows denote Mutual Trust, and a blue icon shows SAP BTP Service. There's also a note about Foundation Models being partner hosted or SAP hosted.
- The diagram level is L2 with a note stating that this BTP Solution Diagram describes how to seamlessly combine various Large Language Models (LLMs) using the Generative AI Hub in SAP AI Core.

Overall, the diagram presents a high-level architecture for integrating user-facing applications with backend AI services and foundation models on SAP's cloud platform.

# Extracting text from images
Nora loves bananabread and thinks recipes are a good example of how LLMs can also extract complex text from images, like from [a picture of a recipe of a bananabread](documents/bananabread.png). Try your own recipe if you like :)

This exercise also shows how you can use the output of an LLM in other systems, as you can tell the LLM how to output information, for example in JSON format.

In [6]:
# get the image from the documents folder
with open("documents/bananabread.png", "rb") as image_file:
    image_data = base64.b64encode(image_file.read()).decode('utf-8')

messages = [{"role": "user", 
             "content": [
                {"type": "text", 
                "text": "Extract the ingredients and instructions in two different json files."},
                {"type": "image_url", 
                 "image_url": {
                    "url": f"data:image/png;base64,{image_data}"
                    }
                }
            ]
        }]

kwargs = dict(model_name=MODEL_NAME, messages=messages)
response = chat.completions.create(**kwargs)

display(Markdown(response.choices[0].message.content))

Here are the extracted contents in two separate JSON files:

**ingredients.json**
```json
{
  "Dry Ingredients": {
    "All-purpose Flour": "260 g",
    "Sugar": "200 g",
    "Baking Soda": "6 g",
    "Salt": "3 g"
  },
  "Wet Ingredients": {
    "Banana": "225 g",
    "Large Eggs": "2",
    "Vegetable Oil": "100 g",
    "Whole Milk": "55 g",
    "Vanilla Extract": "5 g"
  },
  "Toppings": {
    "Chocolate Chips": "100 g",
    "Walnuts (Optional)": "100 g"
  }
}
```

**instructions.json**
```json
{
  "Directions": [
    "Preheat oven to 180 °C",
    "Mash banana in a bowl",
    "Combine banana and other wet ingredients together in the same bowl",
    "Mix all dry ingredients together in a separate bowl",
    "Use a whisk to combine dry mixture into wet mixture until smooth",
    "Pour mixture into a greased/buttered loaf pan",
    "Place the pan into preheated oven on the middle rack and bake for about 60 minutes or until a toothpick comes out clean",
    "Enjoy!"
  ]
}
```

[Next exercise](04-create-embeddings.ipynb)
